In [ ]:
 pip install nltk

In [ ]:
pip install newspaper3k

In [ ]:
#import libraries
from newspaper import Article

import random
import string
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings
import pandas as pd 

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from IPython import display
from nltk.corpus import PlaintextCorpusReader
warnings.filterwarnings('ignore')
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [ ]:
nltk.download('punkt',quiet=True)

In [ ]:
#pre processing data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

In [ ]:
example1 = "/content/agricitease1.txt"
file1 = open(example1, "r")
FileContent = file1.read()

print(FileContent)

In [ ]:
text = FileContent
agricitease_list = nltk.sent_tokenize(text)
print(agricitease_list)

In [ ]:
agricitease_list1 = ['agricitease','farmer','consumer','representative','delivery timmigs','delivery','features','farmer features','consumer features','representative features','web application','website']

In [ ]:
!pip install fontstyle --upgrade
import fontstyle

In [ ]:
nltk.download('punkt',quiet=True)

In [ ]:
article = Article('https://agritech.tnau.ac.in/agricultural_marketing/agrimark_India.html')
article.download()
article.parse()
article.nlp()
agri = article.text

In [ ]:
print(agri)


In [ ]:
#tokenization
text = agri
sentence_list = nltk.sent_tokenize(text) 


In [ ]:
print(sentence_list)
print(agricitease_list)

In [ ]:
# function to return random greeting response
def greeting_response(text):
  text = text.lower()

  #bots greetinng response
  bot_greetings = ['Hi there, how can we help you today?','Hi, Good to see you, do you need something?','Hey! Can I help you with something?','Hey! I am glad that you are here. Please tell me, how can I help you?']
  #users greetings
  user_greetings= ['hi','hellooo','hello','greetings','wassup']

  for word in text.split():
    if word in user_greetings:
      return random.choice(bot_greetings)

In [ ]:
# function to return random goodbye response
def goodbye_response(text):
  text = text.lower()

  #bots goodbye response
  bot_goodbye = ['Bye!Have a good day!','Goodbye','Come visit us again.Bye-Bye!','See you later!']
  #users goodbye
  user_goodbye= ['byee','goodbye','adios','bye']

  for word in text.split():
    if word in user_goodbye:
      return random.choice(bot_goodbye)

In [ ]:
#funtion to return about agrigobot
def who_response(text):
  text = text.lower()

  #bots response
  bot_whoresponse = ['Iam your very own AgriGo bot, here to help!',"I'm AgriGo bot, here to assist you!"]
  #users ques
  user_ques =['who are you?','what is your name?', 'what do you do?']


  if text in user_ques:
    return random.choice(bot_whoresponse)


In [ ]:
# function to return random thank-you response
def thanks_response(text):
  text = text.lower()

  #bots thanks response
  bot_thanks = ['You got it','Don’t mention it','No worries','Not a problem','My pleasure','It was nothing','I’m happy to help','Anytime']
  #users thanks
  user_thanks= ['Thanks so much','Thanks a lot','Thanks a bunch','Thanks a ton', 'Thanks','thanks','thank you','thankyou']

  for word in text.split():
    if word in user_thanks:
      return random.choice(bot_thanks)

In [ ]:
# function to return for unmatched statements
def idk_response(text):
  text = text.lower()

  #bots idk response

  bot_idk = ["What did you say?","I don't understand.","Excuse me, I didn't get it.","Excuse me, can you please repeat it?","Sorry, I did not catch that.","I missed that.","I don't get it.","I’m afraid it is not clear what you saying.","I am sorry, but I don’t follow what you are saying.","I don’t catch what you said.Sorry."]

  return random.choice(bot_idk)

In [ ]:
def agricitease_response(user_input):

  user_input = user_input.lower()
  user_input1= user_input.split(" ")
  for word in user_input1:
    if word in agricitease_list1:
      agricitease_list.append(word)
      agricitease_response = ''
      cm = CountVectorizer().fit_transform(agricitease_list)
      similarity_scores = cosine_similarity(cm[-1],cm)
      similarity_scores_list = similarity_scores.flatten()
      index = index_sort(similarity_scores_list)
      index = index[1:]
      response_flag = 0
      j = 0
      for i in range(len(index)):
        if similarity_scores_list[index[i]] > 0.0:
          colour = '\33[34m'
          agricitease_response = (fontstyle.apply('\33[34m' "Agri bot: "  + agricitease_list[index[i]],'bold/Italic'))
          response_flag = 1
          j = j+1
          if j >2:
            break
          if response_flag == 0:
            color = '\33[32m'
            agricitease_response = (fontstyle.apply('\33[32m' "Agri bot: " +idk_response(user_input),'bold/Italic'))
            agricitease_list.remove(user_input)
      return agricitease_response


In [ ]:
def index_sort(list_var):
  length = len(list_var)
  list_index = list(range(0, length))

  x = list_var
  for i in range(length):
    for j in range(length):
      if x[list_index[i]] > x[list_index[j]]:
        #swap
        temp = list_index[i]
        list_index[i] = list_index[j]
        list_index[j] = temp

  return list_index

In [ ]:
# create bot response
def bot_response(user_input):
  user_input = user_input.lower()
  sentence_list.append(user_input)
  bot_response = ''
  cm = CountVectorizer().fit_transform(sentence_list)
  similarity_scores = cosine_similarity(cm[-1],cm)
  similarity_scores_list = similarity_scores.flatten()
  index = index_sort(similarity_scores_list)
  index = index[1:]
  response_flag = 0

  j = 0
  for i in range(len(index)):
    if similarity_scores_list[index[i]] > 0.0:
      colour = '\33[34m'
      bot_response = (fontstyle.apply('\33[34m' "Agri bot: "  + sentence_list[index[i]],'bold/Italic'))
      response_flag = 1
      j = j+1
    if j >2:
      break

  if response_flag == 0:
    color = '\33[32m'
    bot_response = (fontstyle.apply('\33[32m' "Agri bot: " + idk_response(user_input),'bold/Italic'))

  sentence_list.remove(user_input)
   
  return bot_response




In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
#start chat
Red_Font = '\33[31m'
print(Red_Font + fontstyle.apply("\033[3mAgri Bot: I am Agri bot. I will answer your queries. If u want to exit please type: bye",'bold/Italic'))


while(True):
  user_input = input()
  if goodbye_response(user_input) != None  :
    print(fontstyle.apply('\33[33m' "Agri bot: "+goodbye_response(user_input),'bold/Italic'))
    break
  else:
    if greeting_response(user_input) != None :

      print(fontstyle.apply('\33[36m' "Agri bot: "+ greeting_response(user_input),'bold/Italic'))
    elif agricitease_response(user_input) != None :
      print(fontstyle.apply( agricitease_response(user_input),'bold/Italic'))


    elif who_response(user_input) != None:
      print(fontstyle.apply('\33[36m' "Agri bot: "+ who_response(user_input),'bold/Italic'))
    elif thanks_response(user_input) != None:
      print(fontstyle.apply('\33[36m' "Agri bot: "+ thanks_response(user_input),'bold/Italic'))
   
    elif:
      
      print(fontstyle.apply('\33[36m' "Agri bot: "+ chatbot_response(user_input),'bold/Italic'))
    else:
      print(fontstyle.apply('\33[36m' "Agri bot: "+ bot_response(user_input),'bold/Italic'))
display.Image("logo.jpg",width=300,height=300)



Agri Bot: I am Agri bot. I will answer your queries. If u want to exit please type: bye
who are you?
Agri bot: I'm AgriGo bot, here to assist you!
what is agricitease
Agri bot:  "AGRICITEASE" affinity founding between Farmers and Consumers
 is an all-inclusive digital platform designed to assist farmers in selling their products The new move will eliminate the need for farmers to sell their products through retailers that will escalate in farmer's profits.
what is your name?
Agri bot: I'm AgriGo bot, here to assist you!
are you an idiot?
Agri bot: Hi there, how can I help? bold/Italic
are u mad?
Agri bot: Hi there, how can I help? bold/Italic
u r waste
Agri bot: Good to see you again bold/Italic
where do u live
Agri bot: Hello, thanks for asking bold/Italic


In [ ]:
from google.colab import drive
drive.mount('/content/drive')